# PyLM - Python Landscape Mosaic model

---
<div>This is a Python implementation of the Landscape Mosaic approach as originaly proposed in <a href='https://ies-ows.jrc.ec.europa.eu/gtb/GTB/psheets/GTB-Pattern-LM.pdf' target='_blank'>Vogt et al. (2024)</a> and further refined in <a href='https://doi.org/10.1371/journal.pone.0304215' target='_blank'>Vogt et al. (2024)</a> to support landscape anaylsis.</div>
<br>Author(s): <a href='https://www.unige.ch/envirospace/people/giuliani' target='_blank'>Gregory Giuliani</a>
<br>Version: 0.1
<br>Date: 2025-04-13
<br>Supported by: SNF DynamicLand; Horizon-Europ Nostradamus; LandShift; and MONALISA

---
<b><u>Outline</u></b>
* [Methdology](#methodology)
* [Intialize](#initialize)
* [Input data](#inputdata)
    * [Clip data](#clip)
* [LM Analysis](#lmAnalysis)
    * [LM Background](#lmBackground)
    * [LM Diversity](#lmDiversity)
    * [LM Agriculture](#lmAgriculture)
    * [LM Natural](#lmNatural)
    * [LM Developed](#lmDeveloped)
    * [LM Anthropic Intensity](#lmAnthropicIntensity)
    * [Heatmap](#lmHeatmap)

---
<a id="methodology"></a>
## Methodology
<i>from <a href='https://ies-ows.jrc.ec.europa.eu/gtb/GTB/psheets/GTB-Pattern-LM.pdf' target='_blank'>Vogt et al. (2024)</a></i>
<div style="text-align: justify">The Landscape Mosaic is a tri-polar classification of a location accounting for the relative contributions of three prevalent land cover types, i.e., Agriculture, Natural, Developed in the window surrounding that location. The classification model is designed to identify anthropogenic activity (land cover classes falling in the categories Agriculture and Developed) in relation to natural land cover.
The tri-polar classification scheme uses the threshold values of 10%, 60%, and 100% along each axis to partition the tri-polar space into 19 mosaic classes. These threshold values are indicative for the presence (10%), dominance (60%), or uniqueness (100%) of each land cover type. A lower-case letter (a-Agriculture, n-Natural, or d-Developed) in a mosaic class name denotes a respective land cover type proportion of at least 10% but less than 60%; an upper-class letter (A, N, D) denotes a respective contribution of at least 60% but less than 100%; A letter does not appear if the respective land cover proportion is less than 10%. Locations being composed of a single land cover type only (100%) are found at the corner points of the Mosaic triangle and are labeled with AA, NN, and DD, respectively. With this notation, Dominance is indicated by upper-case letters, an Interface Zone by a combination of upper- and lower-case letters, and a Mixture is indicated by lower-case letters only. The Mosaic colors reflect the varying degree in color intensity with respect to the proportion of blue-Agriculture, green-Natural, and red-Developed.</div>

<div style="text-align: justify">In the resulting LM map, each pixel has a triplet of values showing the relative contribution to the 3 land cover types Agriculture, Natural, Developed. This implies that each pixel value triplet is positioned at a specific location within the triangular domain space. The 19 sub-sections of the triangle - representing presence, dominance and uniqueness - are color- coded into 19 mosaic classes, which are displayed in the resulting LM spatial maps and summarized in the legend above. Because each image pixel value triplet corresponds to a specific location in the triangle, the entire set of image pixels can be inserted in the triangle, resulting in a heatmap (point cloud distribution). To minimize computation time and facilitate the interpretation, this process is conducted for the 100 sub-triangles only, defined by the 10% intervals along each axis. The corner points of the triangle representing exclusive presence of one land cover type only form an additional 3 classes. With this setup, the heatmap consist of 103 occurrence classes, showing the relative pixel occurrence frequency in each sub-space.</div>

---
<a id="initialize"></a>
## Initialize
<div style="text-align: justify">This module first test if the necessary librairies are installed and then user is able to define both input and output folders.</div>

In [1]:
# Test if the libraries are installed and if not install them
try:
  import rasterio
except:
  !pip install rasterio
  import rasterio

try:
  import numpy
except:
  !pip install numpy
  import numpy

try:
  import scipy
except:
  !pip install scipy
  import scipy

try:
  import csv
except:
  !pip install csv
  import csv

#Import all necessary libraries
import rasterio
import numpy as np
import scipy
import rasterio.mask
from matplotlib import pyplot
import matplotlib.pyplot as plt
from rasterio.plot import show
from itertools import product
from rasterio.transform import Affine
from rasterio.transform import from_origin
from numpy.lib.stride_tricks import sliding_window_view
import csv

In [2]:
#Define the global variables
inputFolder = '../inputs/' #To be adapted by user
outputFolder = '../outputs/' #To be adapted by user

<a id="inputdata"></a>
## Input Data - Import the 3-class Land Cover map
<div style="text-align: justify">The input image for the Landscape Mosaic analysis must be a raster map with no more than 3 target classes having the assignment AND (1 Byte - Agriculture, 2 Byte - Natural, 3 Byte - Developed) plus an optional class value of 0 Byte which is reserved for masking missing/no-data pixels. The target classes must not necessary be Agriculture, Natural, Developed but could be any three prevalent land cover types in a given area.</div>

In [ ]:
#read the LCCS Level-3 geotiff file
l3 = rasterio.open(inputFolder+'level3_out_ch_2018.tif') #To be adapted by user

#Print different information on the layer
#Dimensions
print('Width: '+str(l3.width))
print('Height: '+str(l3.height))

#CRS
print('CRS: '+str(l3.crs))

# Bounds of the file
print('BBOXh: '+str(l3.bounds))

#Number of band
print('Index: '+str(l3.indexes))

#Resolution
print('Resolution: '+str(l3.res))

#Get unique LCCS3 code present in the map
band1 = l3.read(1)
np.unique(band1)

In [ ]:
#Visualize the LCCS Level-3 map
pyplot.imshow(band1, cmap='grey')
pyplot.show()  

<a id="lmAnalysis"></a>
## LM Analysis
<i>from <a href='https://ies-ows.jrc.ec.europa.eu/gtb/GTB/psheets/GTB-Pattern-LM.pdf' target='_blank'>Vogt et al. (2024)</a> and <a href='https://publications.jrc.ec.europa.eu/repository/handle/JRC120383' target='_blank'>Maes et al. (2020)</a></i>
<div style="text-align: justify">Each pixel in the LM-image is derived by a moving window procedure in the following way: a) a fixed-area square window is centered over a given pixel of the land cover type image; b) the composition of the three land cover types (the contribution triplet in A, N, D) is calculated from the pixels covered by the square window; c) the corresponding mosaic class is placed on the LM-image at the location of the investigated pixel. This implies that each pixel in the LM-image describes the land cover context within the surrounding square area.</div>
<div style="text-align: justify">The full LM-information is then summarised into the following five stratification layers with specific focus on dedicated thematic topics: 

1. LM-Background: LM summary into 4 classes: Natural, Agriculture, Developed and Mixed.  

2. LM-Diversity: LM summary into 4 classes showing increasing degree of land cover diversity from Uniform, Dual, Triple, to Intermixed land cover.  

3. LM-Agriculture: LM interface summary into 3 classes showing areas where agricultural land cover is dominant (>= 60%), subdominant, or minor (<10%).  

4. LM-Natural: LM interface summary into 3 classes showing areas where natural land cover is dominant (>= 60%), subdominant, or minor (<10%).  

5. LM-Developed: LM interface summary into 3 classes showing areas where developed land cover is dominant (>= 60%), subdominant, or minor (<10%).

<div>
<div style="text-align: justify">The scope of the LM-Background layer is to facilitate the reporting on land cover composition by focusing on the dominant presence (>= 60%) of each of the three land cover types. The second stratification layer, LMDiversity, reports on the degree of spatial land cover heterogeneity. Because land cover dynamics are mainly driven by human activities it is of interest to investigate the interface zones for each of the 3 land cover types with their surrounding neighbourhood. Hence, the purpose of the stratification layer 4 (LM-Natural) is to delineate areas with prevalent natural land cover from those impacted by anthropogenic activities. The purpose of the stratification layers 3 (LM-Agriculture) and 5 (LM-Developed) is to locate and show the intensity of the human footprint on the landscape originating from Agriculture and Developed pressure, respectively. The mapping of the three interface zones (stratification layer 3-5) is an essential prerequisite for policy planning, monitoring and assessment, and towards understanding potential impacts of anthropogenic activities on the environment.</div>
    
<div style="text-align: justify">Because each pixel has a contribution triplet in A, N, D, it will fall in one of the 103 sub-spaces of the LM-triangle. The heatmap shows the relative occurrence frequency of all LM-image pixels. Because occurrence frequencies are shown in percent, the heatmap can be used to a) compare landscape composition for images of different extent, or b) investigate time series at the same observation scale, or c) investigate changes in land cover prevalence across different scales.</div>

### Moving Window count
Computes the frequencies for the three end-members and store them into a 3D-array

In [ ]:
#General idea: iterate over the entire array
#for each pixel > store c1-c2-c3 (values store in a 3-band raster, each representing one class
#the produced raster is then used for the Map - Heatmap - LM_Background, ...
#Padding: https://www.pythoncentral.io/how-to-use-numpy-pad-examples-and-syntax/
#if the LCCS3 map is at 10m resolution then a 10x10 window will give you a direct %

#window size
res=10

# Define the window shape
window_shape = (res, res)

# Padding size to ensure all pixels are included
pad_width = res // 2

# Pad the input 2D array to ensure that all pixels of the country are processed
padded_raster = np.pad(raster, pad_width, mode='edge')

# Create a sliding window view of the raster with shape
windows = np.lib.stride_tricks.sliding_window_view(padded_raster, window_shape)

# Initialize an array to store the result
result = np.empty((3, windows.shape[0], windows.shape[1]), dtype=raster.dtype)

# Iterate over the windows
for i in range(windows.shape[0]):
    for j in range(windows.shape[1]):

        # Extract the current 10x10 window
        window = windows[i, j]

        # Calculate the sum and store it in the result array
        result[0, i, j] = np.count_nonzero(window == 1)  # Band 1: Agriculture count
        result[1, i, j] = np.count_nonzero(window == 2)   # Band 2: Natural count
        result[2, i, j] = np.count_nonzero(window == 3)   # Band 3: Developed count

# Save the result as a new GeoTIFF file
output_filename = outputFolder+"lmCount.tif"

# Define GeoTIFF metadata, you may need to adjust these depending on your data 
transform = from_origin(src.bounds.left, src.bounds.top, res, res)  # transform (origin x, origin y, pixel size x, pixel size y)
new_dtype = raster.dtype

with rasterio.open(
    output_filename,
    "w",
    driver="GTiff",
    height=result.shape[1],
    width=result.shape[2],
    count=3,  # Three bands for mean, min, and max
    dtype=new_dtype,
    crs="EPSG:2056",  # Coordinate reference system; adjust as needed
    transform=transform,
) as dst:
    dst.write(result[0], 1)  # Write Band 1 (agriculture)
    dst.write(result[1], 2)  # Write Band 2 (natural)
    dst.write(result[2], 3)  # Write Band 3 (developed)

print(f"GeoTIFF file '{output_filename}' created successfully.")

---
<br>License: https://creativecommons.org/licenses/by/4.0/ 
<br><img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by.png" alt="CC-BY" width="100"/>